In [2]:
from preparation import do_the_job
data = do_the_job(path="data/data.csv")
X = data['input']
Y = data['output']

X.describe()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,nr.employed,job_blue-collar,job_entrepreneur,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
count,41188.00000,41188.000000,41188.000000,41188.000000,4.118800e+04,4.118800e+04,4.118800e+04,4.118800e+04,41188.000000,41188.000000,...,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,2.567593,962.475454,0.172963,7.797546e-17,-3.315820e-14,-1.352495e-16,-1.024860e-14,0.224677,0.035350,...,0.334296,0.099568,0.017432,0.013839,0.206711,0.209357,0.196416,0.197485,0.863431,0.033335
std,10.42125,2.770014,186.910907,0.494901,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,0.417375,0.184665,...,0.471750,0.299427,0.130877,0.116824,0.404951,0.406855,0.397292,0.398106,0.343396,0.179512
min,17.00000,1.000000,0.000000,0.000000,-2.216433e+00,-2.374889e+00,-2.224953e+00,-2.815697e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.00000,1.000000,999.000000,0.000000,-1.197935e+00,-8.649547e-01,-4.747909e-01,-9.402810e-01,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,38.00000,2.000000,999.000000,0.000000,6.480923e-01,2.994571e-01,-2.803284e-01,3.316799e-01,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,47.00000,3.000000,999.000000,0.000000,8.390606e-01,7.227225e-01,8.864466e-01,8.451701e-01,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,98.00000,56.000000,999.000000,7.000000,8.390606e-01,2.058168e+00,2.939106e+00,8.451701e-01,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Hyperparamètres et nombre de folds pour la validation croisée
param_grid = {
    'max_depth': [3,4,5,6],
    'loss': ['log_loss','exponential'],
    'n_estimators': [50,100,200,250],
    'learning_rate': [0.5,0.7,0.9],
    'subsample': [0.5,1,1.5]
}
num_folds = 5

# Définition de la métrique d'évaluation
scoring = 'precision'

gb = GradientBoostingClassifier(random_state=42)

# Recherche sur grille avec validation croisée
grid_search = GridSearchCV(
    estimator=gb,
    param_grid=param_grid,
    cv=cv,
    scoring=scoring,
)
grid_search.fit(X_train, y_train)

# Les meilleurs hyperparamètres
best_params = grid_search.best_params_
print("Meilleurs hyperparamètres : ", best_params)
best_gb = GradientBoostingClassifier(random_state=42, **best_params)

In [ ]:
from sklearn.model_selection import cross_val_score

# Zoom sur la CV

#Recall de chaque fold
recall_scores = cross_val_score(best_gb, X_train, y_train, cv=num_folds, scoring='recall')

# Affiche les scores pour chaque fold
for i, score in enumerate(recall_scores):
    print(f"Recall pour le fold {i + 1}: {score}")

#Accuracy de chaque fold
accuracy_scores = cross_val_score(best_gb, X_train, y_train, cv=num_folds, scoring='accuracy')

# Affiche les scores pour chaque fold
print("\n")
for i, score in enumerate(accuracy_scores):
    print(f"Accuracy pour le fold {i + 1}: {score}")
    
#Precision de chaque fold
precision_scores = cross_val_score(best_gb, X_train, y_train, cv=num_folds, scoring='precision')

# Affiche les scores pour chaque fold
print("\n")
for i, score in enumerate(precision_scores):
    print(f"Precision pour le fold {i + 1}: {score}")

In [ ]:
# Entraîner le modèle final sur la base entière
best_gb.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = best_gb.predict(X_test)

precision = metrics.precision_score(y_test, y_pred)
print(f"Précision : {precision}")

In [ ]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
results = {
    'True': [tp, fn],
    'False': [fp, tn]
}
results = pd.DataFrame(data, index=['Predicted True', 'Predicted False'])
results

# Extreme Gradient Boosting Classifier

In [15]:
import xgboost as xgb

param_grid = {
    "max_depth": [2, 4, 6],  # Largeur des arbres
    "n_estimators": [25, 50, 100],  # Nombre d'arbres
    "learning_rate": [0.1, 0.2, 0.5, 0.7],  # Taux d'apprentissage
    "subsample": [0.3, 0.5, 0.7, 0.9],  # Ratio de sous-échantillonnage
}

xgb_model = xgb.XGBClassifier(eval_metric="error")
clf = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    cv=cv,
)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

0.9000729027410312
{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 25, 'subsample': 0.7}


In [16]:
best_xgb_model = clf.best_estimator_
best_xgb_model.fit(X_train, y_train)
y_pred = best_xgb_model.predict(X_test)

In [17]:
precision = metrics.precision_score(y_test, y_pred)
print(f"Précision : {precision}")

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
results = {
    'True': [tp, fn],
    'False': [fp, tn]
}
results = pd.DataFrame(results, index=['Predicted True', 'Predicted False'])
results

Précision : 0.709832134292566


,True,False
Predicted True,296,121
Predicted False,1096,10844
